In [124]:
import sqlite3
import csv
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Lipinski
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Crippen
import numpy as np

In [125]:
conn = sqlite3.connect('sabs_moonshot.db')

# Extract the compounds with missing descriptors

In [126]:
df = pd.read_sql_query("""
SELECT 
    compounds.id,
    compounds.MW,
    rdkit_descriptors.HBA,
    rdkit_descriptors.HBD, rdkit_descriptors.cLogP,
    compounds.smiles
FROM rdkit_descriptors 
INNER JOIN compounds on compounds.id = rdkit_descriptors.compound_id
WHERE HBA = "" OR HBD = "" OR MW = "" OR cLogP = "" AND assayed = TRUE;
""",
conn)
print(df)

                     id MW HBA HBD cLogP  \
0    DAR-DIA-6a508060-1                    
1    DAR-DIA-6a508060-2                    
2    DAR-DIA-6a508060-3                    
3    DAR-DIA-6a508060-4                    
4    DAR-DIA-6a508060-5                    
5    DAR-DIA-6a508060-7                    
6    DAR-DIA-6a508060-8                    
7    DAR-DIA-6a508060-9                    
8   DAR-DIA-6a508060-10                    
9   DAR-DIA-6a508060-11                    
10  DAR-DIA-6a508060-12                    
11  DAR-DIA-6a508060-13                    
12  DAR-DIA-6a508060-14                    
13  DAR-DIA-6a508060-15                    
14  DAR-DIA-6a508060-16                    
15   ALP-UNI-b33a865d-1                    
16   ALP-UNI-b33a865d-1                    
17   ALP-UNI-b33a865d-1                    
18   DAR-DIA-6a508060-3                    
19   DAR-DIA-6a508060-4                    
20   PRA-UNK-2c426785-1                    
21   EDG-MED-0c930815-1         

# Using RDKit to compute molecular descriptors and Lipinski's rule violations

In [127]:
class Molecule:
    """ A molecule. In particular, either the R1 or R2 group, or the scaffold
    and one or two groups.
    There are methods which tell you the properties of the molecule and if it
    passes the Lipsinki test
    """

    def __init__(self, mol_smiles):
        """Constructor for Molecule class. Initialises Molecule instance from
        smile string.
        :param mol_smiles: smile string of molecule
        :type mol_smiles: String
        """
        self.__mol_smiles = mol_smiles

    # @property
    # def get_smile_string(self):
    #     """Returns molecule's smile string
    #     :return: smile string of molecule
    #     :rtype: String
    #     """
    #     return self.__mol_smiles

    def descriptors(self):
        """Calculate molecule descriptor metrics as dict:
        | mol - smile string
        | MW - molecular weight
        | logP - logP
        | TPSA - topological polar surface area
        | HA - heavy atom count
        | h_acc - H acceptor count
        | h_don - H donator count
        | rings - ring count
        :return: molecule descriptor metrics
        :rtype: dict
        """
        mol = Chem.MolFromSmiles(self.__mol_smiles)
        mw = Descriptors.ExactMolWt(mol)
        log_p = Crippen.MolLogP(mol)
        tpsa = rdMolDescriptors.CalcTPSA(mol)  # topological polar surface area
        ha = Lipinski.HeavyAtomCount(mol)  # heavy atom count
        h_acceptors = Lipinski.NumHAcceptors(mol)
        h_donors = Lipinski.NumHDonors(mol)
        rings = Lipinski.RingCount(mol)
        molar_refractivity=Chem.Crippen.MolMR(mol)
        rotatable_bonds= Descriptors.NumRotatableBonds(mol)
        desc_dict = {'smiles': self.__mol_smiles, #changed from mol to smiles so consistent with database we downloaded
                     'MW': mw,
                     'logP': log_p,
                     'TPSA': tpsa,
                     'HA': ha,
                     'h_acc': h_acceptors,
                     'h_don': h_donors,
                     'rings': rings,
                     'MR': molar_refractivity, 
                     'Rot_bonds': rotatable_bonds
                     }
        return desc_dict

    def lipinski(self, desc_dict):
        """Calculate Lipinski from the descriptor dictionary. Returns the
        number of rules broken and whether the molecule passes.
        :param desc_dict: molecule descriptor metrics
        :type desc_dict: dict
        :return: violations
        :rtype: dict
        """
        violations = {'MW': desc_dict['MW'] < 500.0,
                      'h_acc': desc_dict['h_acc'] <= 10,
                      'h_don': desc_dict['h_don'] <= 5,
                      'logP': desc_dict['logP'] < 5}
        return violations

Select smiles strings from database

In [128]:
df = pd.read_sql_query("""
SELECT compounds.id, compounds.smiles
FROM compounds 
INNER JOIN assays ON compounds.id = assays.compound_id
WHERE assays.r_avg_IC50 != "" AND assays.f_avg_IC50 != "" ;
""",
conn)
df

,id,smiles
0,DAR-DIA-23aa0b97-19,N#Cc1cccc(NC(=O)Cc2cncc3ccccc23)c1
1,DAR-DIA-23aa0b97-20,O=C(Cc1cncc2ccccc12)Nc1ccccc1
2,TRY-UNI-714a760b-3,Cc1c(N)cncc1NC(=O)Cc1cccc(Cl)c1
3,TRY-UNI-714a760b-6,Cc1ccncc1NC(=O)Cc1cccc(Cl)c1
4,TRY-UNI-714a760b-12,Cc1ccncc1NC(=O)Nc1cccc(Cl)c1
...,...,...
656,WAR-XCH-79d12f6e-6,Cc1ccc(C)c(S(=O)(=O)N2CCN(C(=O)CCl)CC2)c1
657,ALP-POS-869ac754-1,O=C(Nc1cncc2ccccc12)C1CCOc2cc(Cl)c(Cl)cc21
658,MED-COV-4280ac29-15,O=C(CCl)N1CCN(Cc2cccc(Cl)c2)CC1
659,LON-WEI-8f408cad-4,O=C(CCl)N1CCN(S(=O)(=O)c2cccc(F)c2)CC1


# Converting compounds database to pandas dataframe

In [129]:
# df = pd.read_sql_query("""
# SELECT *
# FROM compounds;
# """,
# conn)
# print(df[:10])

In [130]:

# new_df=pd.DataFrame()
# descriptors={'a': 65, 'b': 66}
# new_df=new_df.append(descriptors, ignore_index=True)
# print(new_df)

In [131]:
# for i in range(len(df)):
#     cur_smile=df.iloc[i]['smiles']
#     print(cur_smile)
#     cur_mol=Molecule(cur_smile)
#     cur_mol.descriptors

# Creating dataframe with smiles and molecular descriptors

In [132]:
# new_df=pd.DataFrame()
desc_dict = [
                     'MW' ,
                     'logP' ,
                     'TPSA' ,
                     'HA' ,
                     'h_acc',
                     'h_don',
                     'rings',
                     'MR',
                     'Rot_bonds'
]

for desc in desc_dict:
    df[desc]=np.nan
df["lipinski"]=np.nan

for i in range(len(df)): #change to range(len(df))
    cur_smile=df.iloc[i]['smiles']
    # print(cur_smile)
    cur_mol=Molecule(cur_smile)
    descriptors=cur_mol.descriptors()
    lipinski=cur_mol.lipinski(descriptors)
    for 
    df["lipinski"].iloc[i]=
    # print(descriptors)
    for key in desc_dict:
        df[key].iloc[i]=descriptors[key]
    

    # df["test"].iloc[i]= 2
    


df.to_csv('./compounds-featurized.csv')
df

SyntaxError: invalid syntax (<ipython-input-132-59a610169b59>, line 24)

# Creating dataframe with molecular descriptors and finding the Lipinski's rule of five violations for each compound

In [ ]:
new_df=pd.DataFrame()
for i in range(2): #change to range(len(df))
    cur_smile=df.iloc[i]['smiles']
    cur_mol=Molecule(cur_smile)
    descriptors=cur_mol.descriptors()
    lipinski=cur_mol.lipinski(descriptors)
    print(lipinski)
    new_df=new_df.append(descriptors, ignore_index=True)
    
    new_df=new_df.append(lipinski, ignore_index=True)

print(new_df)



{'MW': True, 'h_acc': True, 'h_don': True, 'logP': True}
{'MW': True, 'h_acc': True, 'h_don': True, 'logP': True}
     HA       MR          MW  Rot_bonds   TPSA  h_acc  h_don     logP  rings  \
0  22.0  85.2337  287.105862        3.0  65.78    3.0    1.0  3.28768    3.0   
1   NaN      NaN    1.000000        NaN    NaN    1.0    1.0  1.00000    NaN   
2  20.0  80.5187  262.110613        3.0  41.99    2.0    1.0  3.41600    3.0   
3   NaN      NaN    1.000000        NaN    NaN    1.0    1.0  1.00000    NaN   

                               smiles  
0  N#Cc1cccc(NC(=O)Cc2cncc3ccccc23)c1  
1                                 NaN  
2       O=C(Cc1cncc2ccccc12)Nc1ccccc1  
3                                 NaN  


# Trying to discard compounds that fail Lipinski's rule of 5

In [ ]:
lipinski_list = ['MW', 'logP', 'h_acc', 'h_don']
    # r_group_1_id = request.args.get('r1')
    # r_group_2_id = request.args.get('r2')
    # molecule_key = tuple2str((r_group_1_id, r_group_2_id))
    # drug_mol = FinalMolecule(r_group_1_id, r_group_2_id)
    # drug_lipinski = drug_mol.lipinski(drug_mol.descriptors())
    # lipinski_dict = {cur_mol: lipinski}
for label in lipinski_list:
    if "False" not in lipinski:
        pass
    # else:
    #     # delete smiles/compound row

# Trying to discard compounds that violate more than one of Lipinski's rule of 5

In [ ]:

# for label in lipinski_list:
#     if "False" appears twice  in lipinski:
#         #delete it
#     # else:
#         pass

# Variants of Lipinski's rule of 5

# Molar refractivity from 40 to 130, 10 or fewer rotatable bonds, Polar surface area no greater than 140 Å2. Drugs are predicted to have predicted to have good oral bioavailability when the last 2 of these are satisfied.

# TPSA is calculated above

# Not calculating these descriptors - not sure why

In [ ]:
mol1=Molecule("OCC")
descriptors=mol1.descriptors()
lipinski=mol1.lipinski(descriptors)
print(lipinski)
h_acceptors = Lipinski.NumHAcceptors(mol1)
print(h_acceptors)
molar_refractivity=Chem.Crippen.MolMR(mol1)
rotatable_bonds= Descriptors.NumRotatableBonds(mol1)
print(molar_refractivity)
print(rotatable_bonds)


{'MW': True, 'h_acc': True, 'h_don': True, 'logP': True}


ArgumentError: Python argument types in
    rdkit.Chem.rdMolDescriptors.CalcNumHBA(Molecule)
did not match C++ signature:
    CalcNumHBA(RDKit::ROMol mol)